In [29]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Preliminary functions
Functions that make our life easier, since we will be reusing some code.

In [30]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. 
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [31]:
def onv_bases(N, K, transition = "detachment"):
    """
    Create two ONV bases, based on whether we're describing an electron attachment or detachment.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    if transition == "detachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha - 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    elif transition == "attachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    else:
        print("Specify a valid electron transition process: attachment/detachment.")
        return None
    
    return (onv_basis_I, onv_basis_J)

In [32]:
def uFCI(molecule, onv_basis):
    """
    FCI calculations for an unrestricted spin-orbital basis.
    """
    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G") # the underlying scalar AO basis is not orthonormal
    spin_orbital_basis.lowdinOrthonormalize() # after using Löwdin's orthonormalizing procedure, the spin orbital basis is now orthonormal
    
    hamiltonian = gqcpy.USQHamiltonian.Molecular(spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

# Preparation of some variables

Generate the neutral molecule with N electrons, a positive ion with N-1 electrons (charge +1) and a negative ion with N+1 electrons (charge -1).

In [51]:
neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/n2.xyz" , 0)  # create a neutral molecule with N electrons
positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/n2.xyz" , +1)  # create a an ion N-1 electrons
negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/n2.xyz" , +1)  # create a an ion with N+1 electrons

#neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule with N electrons
#positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion N-1 electrons
#negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with N+1 electrons

Build an ONV basis for a specified electron transition process, given the dimensions N and K. For this notebook, we have opted for an electron detachment.

Here, $I$ describes the state of the $(N \pm 1)$-electron system $|N\pm1, (I)>$ and $J$ the state of the $N$-electron system $|N, (J)>$.

In [52]:
N, M = dimensions(neutral_molecule)
K = M//2 # number of spinors per spin compound

onv_basis_I, onv_basis_J = onv_bases(N, K, "detachment")

# Dyson orbitals in the uncorrelated limit

Build the linear expansion for a CI wave function in the Hartree-Fock formalism, where all coefficients of a given ONV basis are zero except for the HF determinant.

In [53]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)

Construct Dyson orbitals using the "pseudo overlap" between an N-1 wave function and a N wave function with an annihilated electron in each spinor. Only if the annihilation $\hat{a}_P$ occurs at spinor $\phi_K(\mathbf{r})$, the spinor that is unoccupied in $|N\pm1, (I)>$, the Dyson amplitude $<N-1, (I)|\hat{a}_P|N, (J)>$ will be non-zero. For H2O in STO-3G basis, this will be the fifth $\alpha$ spin-orbital $\phi_{5\alpha}(\mathbf{r})$.

In [54]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In a Hartree-Fock picture, the difference of two electron densities, one of an $N$-electron system and the other of a $(N-1)$-electron system, should be reduced to the norm of a Dyson orbital. For a more profound theoretical explanation, the reader is referred to [this page](https://gqcg-res.github.io/dariatols_knowdes/correlation-in-the-dyson-orbital-picture.html#uncorrelated-limit). 

In [55]:
electron_density_I = np.sum( np.diagonal( Psi_I.calculate1DM() ) )
electron_density_J = np.sum( np.diagonal( Psi_J.calculate1DM() ) )

print("Electron density for |N-1, (I)>:")
print(electron_density_I)

print("\n" + "-"*25 + "\n")

print("Electron density for |N, (J)>:")
print(electron_density_J)

Electron density for |N-1, (I)>:
13.0

-------------------------

Electron density for |N, (J)>:
14.0


In [ ]:
print("Is the difference between the N and (N-1) electron density equal to the Dyson orbital that represents the set of transition amplitudes between these two systems?")
print( electron_density_J - electron_density_I == d_IJ.amplitudes()[6]**2 )

# Dyson orbitals when correlation is present

In contrary to the HF framework, the wave function for both the neutral molecule and the positive ion will now have contributions from more determinants than just the HF determinant.

In [ ]:
Psi_I = uFCI(positive_ion, onv_basis_I)
Psi_J = uFCI(neutral_molecule, onv_basis_J)

Construct Dyson orbital containing transition amplitudes defined as $<N-1, (I)|\hat{a}_P|N, (J)>$.

In [47]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([ 1.53697866e-15, -1.31690415e-14,  1.72273738e-14, -1.92340620e-14,
       -9.43605291e-01,  4.58077473e-15,  4.31068202e-14])

In [48]:
electron_density_I = np.sum( np.diagonal( Psi_I.calculate1DM() ) )
electron_density_J = np.sum( np.diagonal( Psi_J.calculate1DM() ) )

print("Electron density for |N-1, (I)>:")
print(electron_density_I)

print("\n" + "-"*25 + "\n")

print("Electron density for |N, (J)>:")
print(electron_density_J)

Electron density for |N-1, (I)>:
9.000000000000085

-------------------------

Electron density for |N, (J)>:
10.000000000000005


In [49]:
print("Is the difference between the N and (N-1) electron density equal to the Dyson orbital that represents the set of transition amplitudes between these two systems?")
print( electron_density_J - electron_density_I == d_IJ.amplitudes()[4]**2 )

Is the difference between the N and (N-1) electron density equal to the Dyson orbital that represents the set of transition amplitudes between these two systems?
False
